<a href="https://colab.research.google.com/github/samirjhb/samirjhb/blob/main/examen_1_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## INSTALACION DE PAQUETE
%%capture
!pip install pyspark
!pip install pyarrow

In [2]:
##CONEXION DRIVE
from google.colab import drive
drive.mount("/content/gdrive")
gdrive='/content/gdrive/MyDrive/big data/'
segmentacion = gdrive + 'Tabla_segmentacion.csv'
pricing = gdrive + 'Tabla_pricing.csv'

Mounted at /content/gdrive


In [45]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import min as pyspark_min, max as pyspark_max
from pyspark.sql.functions import count, avg, col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import desc


# Crear una sesión de Spark
spark = SparkSession.builder.master("local[*]").appName("Ejemplo").getOrCreate()

# Cargar DataFrames desde archivos CSV
df_seg = spark.read.csv(segmentacion, inferSchema=True, header =True,sep=";")
df_pri = spark.read.csv(pricing, inferSchema=True, header =True,sep=";")

In [31]:
## Punto 1
# Contar registros en cada DataFrame
conteo_seg = df_seg.count()
conteo_pri = df_pri.count()

# Calcular la unión de ambos DataFrames
union_registros = conteo_seg + conteo_pri

# Imprimir resultados
print(f"Registros en Tabla_segmentacion.csv: {conteo_seg}")
print(f"Registros en Tabla_pricing.csv: {conteo_pri}")
print(f"Registros en ambos archivos: {union_registros}")

Registros en Tabla_segmentacion.csv: 2805667
Registros en Tabla_pricing.csv: 12581326
Registros en ambos archivos: 15386993


In [32]:
## Punto 2
# Seleccionar solo las columnas relevantes
df_pri = df_pri.select("FAMILIA", "MARGEN")

# Calcular los rangos mínimos y máximos por familia
resultados_familias = df_pri.groupBy("FAMILIA").agg(
    pyspark_min("MARGEN").alias("min_margen"),
    pyspark_max("MARGEN").alias("max_margen")
)

# Mostrar resultados
resultados_familias.show(truncate=False)

+-------+------------------+------------------+
|FAMILIA|min_margen        |max_margen        |
+-------+------------------+------------------+
|1      |0.1306476481437908|0.6543916706056292|
|10     |0.1306476481437908|0.9348571428571428|
|11     |0.1306476481437908|0.6968634975081676|
|12     |0.1306476481437908|0.6862996158770807|
|13     |0.1306476481437908|0.5705853658536586|
|2      |0.1306476481437908|0.9129213483146068|
|3      |0.1306476481437908|0.5800000000000001|
|4      |0.1306476481437908|0.8433924466193962|
|5      |0.1306476481437908|0.9989324223337248|
|6      |0.1306476481437908|0.9984176477444924|
|7      |0.1306476481437908|0.6961444508475835|
|8      |0.1306476481437908|0.9423213066112972|
|9      |0.1306476481437908|0.7539181286549708|
+-------+------------------+------------------+



In [38]:
##Punto 3

# Contar el número de clientes por segmento en df_seg
clientes_seg = df_seg.groupBy("SEGMENTO").agg(count("RUT_CLI").alias("NUM_CLIENTES_SEG"))

# Contar el número de clientes por segmento en df_pri
clientes_pri = df_pri.groupBy("SEGMENTO").agg(count("FAMILIA").alias("NUM_CLIENTES_PRI"))

# Unir ambos resultados por segmento
clientes_por_segmento = clientes_seg.join(clientes_pri, "SEGMENTO")

# Calcular el promedio de clientes por segmento
clientes_por_segmento = clientes_por_segmento.withColumn("PROMEDIO_CLIENTES", (col("NUM_CLIENTES_SEG") + col("NUM_CLIENTES_PRI")) / 2)

# Imprimir resultados
clientes_por_segmento.show()

+--------+----------------+----------------+-----------------+
|SEGMENTO|NUM_CLIENTES_SEG|NUM_CLIENTES_PRI|PROMEDIO_CLIENTES|
+--------+----------------+----------------+-----------------+
|   45307|               1|             578|            289.5|
|   35912|               1|            1445|            723.0|
|   35982|               1|            2312|           1156.5|
|  124647|               1|             578|            289.5|
|  124967|               1|             578|            289.5|
|  121854|               1|             578|            289.5|
|   33868|               3|            1734|            868.5|
|   36538|               4|            4046|           2025.0|
|  124861|               1|            1156|            578.5|
|   32855|               1|            1734|            867.5|
|  124743|               1|            1156|            578.5|
|   37111|               2|            1156|            579.0|
|   35820|               1|             867|           

In [41]:
## Punto 4

# Contar la cantidad de rubros distintos en df_pri
rubros_count = df_pri.agg(countDistinct("RUBRO").alias("NUM_RUBROS"))

# Mostrar resultados
rubros_count.show()


+----------+
|NUM_RUBROS|
+----------+
|        17|
+----------+



In [46]:
## Punto 5

# Encontrar el cliente con el mayor precio
cliente_mayor_precio = df_pri.orderBy(desc("MARGEN")).first()

# Mostrar resultados
print("Cliente con el mayor precio:")
print(cliente_mayor_precio)

Cliente con el mayor precio:
Row(SEGMENTO=1, SUCURSAL='E523', RUBRO='C5 ', FAMILIA=5, SUBFAMILIA=11, PRODUCTO=None, JERARQUIA=2, MARGEN=0.9989324223337248, MARGEN_PRE_APRO=0.9989324223337248)


In [43]:
## Punto 6

# Calcular el margen promedio por sucursal en df_pri
margen_promedio_por_sucursal = df_pri.groupBy("SUCURSAL").agg(avg("MARGEN").alias("MARGEN_PROMEDIO"))

# Encontrar la sucursal con el mayor margen promedio
sucursal_max_margen = margen_promedio_por_sucursal.orderBy("MARGEN_PROMEDIO", ascending=False).first()

# Encontrar la sucursal con el menor margen promedio
sucursal_min_margen = margen_promedio_por_sucursal.orderBy("MARGEN_PROMEDIO").first()

# Mostrar resultados
print("Sucursal con el mayor margen promedio:")
print(sucursal_max_margen)

print("\nSucursal con el menor margen promedio:")
print(sucursal_min_margen)

Sucursal con el mayor margen promedio:
Row(SUCURSAL='E550', MARGEN_PROMEDIO=0.3260823808111714)

Sucursal con el menor margen promedio:
Row(SUCURSAL='E576', MARGEN_PROMEDIO=0.2753196305799647)


In [ ]:
# Imprimir el esquema del DataFrame df_pri y df_seg
#df_pri.printSchema()
# df_seg.printSchema()
df_pri.show()
df_seg.show()
